In [2]:
!pip -q install -r requirements.txt

In [3]:
import sys
sys.path.append('/anaconda/envs/azureml_py38/lib/python3.8/site-packages')

#importing the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [4]:
#importing the dataset
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
X = data['data']
y = data['target']
print("shape of x: {}\nshape of y: {}".format(X.shape,y.shape))

shape of x: (569, 30)
shape of y: (569,)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=69)

In [6]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
EPOCHS = 500
BATCH_SIZE = 64
LEARNING_RATE = 0.001

In [8]:
## train data
class TrainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


train_data = TrainData(torch.FloatTensor(X_train), 
                       torch.FloatTensor(y_train))
## test data    
class TestData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

test_data = TestData(torch.FloatTensor(X_test))

In [9]:
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [10]:
class BinaryClassification(nn.Module):
    def __init__(self, features_number):
        super(BinaryClassification, self).__init__()
        # Number of input features is passed.
        self.layer_1 = nn.Linear(features_number, 64) 
        self.layer_2 = nn.Linear(64, 64)
        self.layer_out = nn.Linear(64, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [12]:
len(X[0])

30

In [13]:
model = BinaryClassification(len(X[0]))
model.to(device)
print(model)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

BinaryClassification(
  (layer_1): Linear(in_features=30, out_features=64, bias=True)
  (layer_2): Linear(in_features=64, out_features=64, bias=True)
  (layer_out): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [14]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [16]:
model.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

Epoch 001: | Loss: 0.00005 | Acc: 100.000
Epoch 002: | Loss: 0.00010 | Acc: 100.000
Epoch 003: | Loss: 0.00009 | Acc: 100.000
Epoch 004: | Loss: 0.00013 | Acc: 100.000
Epoch 005: | Loss: 0.00003 | Acc: 100.000
Epoch 006: | Loss: 0.00023 | Acc: 100.000
Epoch 007: | Loss: 0.00004 | Acc: 100.000
Epoch 008: | Loss: 0.00003 | Acc: 100.000
Epoch 009: | Loss: 0.00010 | Acc: 100.000
Epoch 010: | Loss: 0.00005 | Acc: 100.000
Epoch 011: | Loss: 0.00016 | Acc: 100.000
Epoch 012: | Loss: 0.00011 | Acc: 100.000
Epoch 013: | Loss: 0.00002 | Acc: 100.000
Epoch 014: | Loss: 0.00002 | Acc: 100.000
Epoch 015: | Loss: 0.00003 | Acc: 100.000
Epoch 016: | Loss: 0.00005 | Acc: 100.000
Epoch 017: | Loss: 0.00005 | Acc: 100.000
Epoch 018: | Loss: 0.00006 | Acc: 100.000
Epoch 019: | Loss: 0.00003 | Acc: 100.000
Epoch 020: | Loss: 0.00002 | Acc: 100.000
Epoch 021: | Loss: 0.00007 | Acc: 100.000
Epoch 022: | Loss: 0.00018 | Acc: 100.000
Epoch 023: | Loss: 0.00003 | Acc: 100.000
Epoch 024: | Loss: 0.00004 | Acc: 

In [42]:
X[0]

array([1.799e+01, 1.038e+01, 1.228e+02, 1.001e+03, 1.184e-01, 2.776e-01,
       3.001e-01, 1.471e-01, 2.419e-01, 7.871e-02, 1.095e+00, 9.053e-01,
       8.589e+00, 1.534e+02, 6.399e-03, 4.904e-02, 5.373e-02, 1.587e-02,
       3.003e-02, 6.193e-03, 2.538e+01, 1.733e+01, 1.846e+02, 2.019e+03,
       1.622e-01, 6.656e-01, 7.119e-01, 2.654e-01, 4.601e-01, 1.189e-01])

In [43]:
X_batch[0]

tensor([-0.1152, -0.3147, -0.1975, -0.1984, -1.2194, -1.1972, -0.8243, -0.8956,
        -0.8758, -1.1265, -0.4829, -0.0973, -0.5380, -0.3511, -1.0614, -1.0820,
        -0.7397, -1.1075, -0.9012, -1.0262, -0.1090,  0.1076, -0.1992, -0.2033,
        -1.1716, -1.0890, -0.7442, -0.9210, -0.9479, -1.1503], device='cuda:0')

In [44]:
model(X_batch)

tensor([[  4.2370],
        [  3.8252],
        [  6.5216],
        [  5.1182],
        [-14.5039],
        [  4.8069],
        [  4.8609],
        [  4.8003],
        [  5.3774],
        [ -9.3326],
        [  6.9971],
        [ -6.0142],
        [  6.3067],
        [  7.8310],
        [  4.4928],
        [-10.7606],
        [  4.7279],
        [  6.4984],
        [  7.5996],
        [  7.4307],
        [  6.9249],
        [  5.1484],
        [ -6.8333],
        [ -7.3271],
        [-11.0650],
        [  7.2275],
        [  7.0271],
        [-10.8509],
        [ -8.7718],
        [ -8.7296],
        [  7.3190],
        [  5.0150],
        [-12.9676],
        [ -8.8649],
        [  6.7360],
        [  7.2112],
        [  5.2709],
        [  7.7155],
        [-11.5559],
        [  5.1562],
        [ -9.3724],
        [  4.4835],
        [  2.1669],
        [  6.0359],
        [  6.6899],
        [  5.0686],
        [  6.1712],
        [  3.3034],
        [-10.0986],
        [-10.4652],
